<center><img src="logo_pret_a_depenser.png">


<center> Notebook 1 - Exploration du jeu de données</center>

# INTRODUCTION

Vous êtes Data Scientist au sein d'une société financière, nommée "Prêt à dépenser", qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.

L’entreprise souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifie la demande en crédit accordé ou refusé. Elle souhaite donc développer un algorithme de classification en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.).

De plus, les chargés de relation client ont fait remonter le fait que les clients sont de plus en plus demandeurs de transparence vis-à-vis des décisions d’octroi de crédit. Cette demande de transparence des clients va tout à fait dans le sens des valeurs que l’entreprise veut incarner.

Prêt à dépenser décide donc de développer un dashboard interactif pour que les chargés de relation client puissent à la fois expliquer de façon la plus transparente possible les décisions d’octroi de crédit, mais également permettre à leurs clients de disposer de leurs informations personnelles et de les explorer facilement. 

# ANALYSE EXPLORATOIRE DES DONNEES

## 1. Importation des données

### 1.1 Importation des bibliothèques 

In [7]:
###importation des bibliothèques : 
### Global : 
import importlib.metadata
import pandas as pd
import numpy as np

### Visualisation : 
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

In [8]:
###versions des bibliothèques : 
!python --version

print('\nfully imported')
print('\n'.join(f'{m.__name__} - {m.__version__}' 
                for m in globals().values() 
                if getattr(m, '__version__', None)))

print('\npartially imported')
#print("deep_translator -", importlib.metadata.version('deep_translator'))
print("importlib -", importlib.metadata.version('importlib.metadata'))
print("matplotlib -", importlib.metadata.version('matplotlib'))
print("numpy -", importlib.metadata.version('numpy'))

Python 3.9.18

fully imported
json - 2.0.9
pandas - 2.1.1
pandas - 2.1.1
numpy - 1.26.4
seaborn - 0.13.1
missingno - 0.5.1

partially imported


PackageNotFoundError: importlib.metadata

In [ ]:
### paramètres des graphiques : 
plt.rcParams['font.size'] = 14

 ### 1.2 Importation des données

Les données sont présentées dans plusieurs datasets suivants : 
- application_{train|test}.csv :
    - Il s'agit de la table principale, divisée en deux fichiers pour Train (avec TARGET) et Test (sans TARGET). Données statiques pour toutes les applications. 
    - Une ligne représente un prêt dans notre échantillon de données.
    
- bureau.csv :
    - Tous les crédits antérieurs du client fournis par d'autres institutions financières qui ont été signalés au bureau de crédit (pour les clients qui ont un prêt dans notre échantillon).
    -  Pour chaque prêt de notre échantillon, il y a autant de lignes que le nombre de crédits que le client avait dans le Credit Bureau avant la date de la demande.
    
- bureau_balance.csv : 
    - Soldes mensuels des crédits antérieurs au bureau de crédit.
    - Ce tableau comporte une ligne pour chaque mois d'historique de chaque crédit précédent signalé au bureau de crédit - c'est-à-dire que le tableau a (nombre de prêts dans l'échantillon * nombre de crédits précédents relatifs * nombre de mois où nous avons un historique observable pour les crédits précédents) lignes.
    
- POS_CASH_balance.csv :
    - Instantanés mensuels du solde des points de vente précédents et des prêts en espèces que le demandeur avait contractés avec Home Credit.
    - Ce tableau a une ligne pour chaque mois d'historique de chaque crédit précédent dans Home Credit (crédit à la consommation et prêts de trésorerie) lié aux prêts de notre échantillon - c'est-à-dire que le tableau a (nombre de prêts dans l'échantillon * nombre de crédits précédents relatifs * nombre de mois dans lequel nous avons un historique observable pour les lignes des crédits précédents).
    
- credit_card_balance.csv :
    - Aperçus mensuels du solde des cartes de crédit précédentes que le demandeur a avec Home Credit.
    - Ce tableau comporte une ligne pour chaque mois d'historique de chaque crédit précédent dans Home Credit (crédit à la consommation et prêts de trésorerie) lié aux prêts de notre échantillon - c'est-à-dire que le tableau a (nombre de prêts dans l'échantillon * nombre de cartes de crédit précédentes relatives * nombre de mois où nous avons un historique observable pour la carte de crédit précédente) lignes.
    
- previous_application.csv :
    - Toutes les demandes précédentes de prêts immobiliers des clients qui ont des prêts dans notre échantillon.
    - Il y a une ligne pour chaque demande précédente liée aux prêts dans notre échantillon de données.     
    
- installments_payments.csv :
    - Historique de remboursement des crédits précédemment décaissés en Home Credit liés aux prêts de notre échantillon.
    - Il y a a) une ligne pour chaque paiement effectué plus b) une ligne pour chaque paiement manqué.
    - Une ligne équivaut à un versement d'une échéance OU une échéance correspondant à un versement d'un précédent crédit immobilier lié aux prêts de notre échantillon.    .
    
- HomeCredit_columns_description.csv :
    - Ce fichier contient les descriptions des colonnes des différents fichiers de données.
<center><img src="home_credit.png">

<a href="https://www.kaggle.com/c/home-credit-default-risk/data">SOURCE DU JEU DE DONNEES</a> 

In [ ]:
###  test pour l'encodage du dataset 'columns_description' :
# Tentative de lecture avec l'encodage ISO-8859-1
try:
    columns_description = pd.read_csv('./HomeCredit_columns_description.csv', encoding='ISO-8859-1')
    print("Fichier lu avec l'encodage ISO-8859-1")
except UnicodeDecodeError:
    # Tentative de lecture avec l'encodage cp1252
    try:
        columns_description = pd.read_csv('./HomeCredit_columns_description.csv', encoding='cp1252')
        print("Fichier lu avec l'encodage cp1252")
    except UnicodeDecodeError:
        # Tentative de lecture avec l'encodage utf-16
        try:
            columns_description = pd.read_csv('./HomeCredit_columns_description.csv', encoding='utf-16')
            print("Fichier lu avec l'encodage utf-16")
        except UnicodeDecodeError as e:
            print("Erreur de décodage:", e)

In [11]:
### Importation données brutes : 

X_train = pd.read_csv('./application_train.csv')
X_test = pd.read_csv('./application_test.csv')
bureau = pd.read_csv('./bureau.csv')
bureau_balance = pd.read_csv('./bureau_balance.csv')
pos_cash = pd.read_csv('./POS_CASH_balance.csv')
credit_card = pd.read_csv('./credit_card_balance.csv')
previous_application = pd.read_csv('./previous_application.csv')
installments_payments = pd.read_csv('./installments_payments.csv')
columns_description = pd.read_csv('./HomeCredit_columns_description.csv', encoding='ISO-8859-1')

 ## 2. Analyse du jeu de données

### 2.1 Taille des datasets

In [31]:
# Calcul du nombre de valeurs NaN pour chaque DataFrame
nan_counts = {
    'X_train': X_train.isna().sum().sum(),
    'X_test': X_test.isna().sum().sum(),
    'bureau': bureau.isna().sum().sum(),
    'bureau_balance': bureau_balance.isna().sum().sum(),
    'pos_cash': pos_cash.isna().sum().sum(),
    'credit_card': credit_card.isna().sum().sum(),
    'previous_application': previous_application.isna().sum().sum(),
    'installments_payments': installments_payments.isna().sum().sum(),
    'columns_description': columns_description.isna().sum().sum()
}

# Création d'un dictionnaire pour faire un récapitulatif de la taille des dataset utilisés et du nombre de NaN
data_summary = {
    'X_train': (X_train.shape[0], X_train.shape[1], nan_counts['X_train']),
    'X_test': (X_test.shape[0], X_test.shape[1], nan_counts['X_test']),
    'bureau': (bureau.shape[0], bureau.shape[1], nan_counts['bureau']),
    'bureau_balance': (bureau_balance.shape[0], bureau_balance.shape[1], nan_counts['bureau_balance']),
    'pos_cash': (pos_cash.shape[0], pos_cash.shape[1], nan_counts['pos_cash']),
    'credit_card': (credit_card.shape[0], credit_card.shape[1], nan_counts['credit_card']),
    'previous_application': (previous_application.shape[0], previous_application.shape[1], nan_counts['previous_application']),
    'installments_payments': (installments_payments.shape[0], installments_payments.shape[1], nan_counts['installments_payments']),
    'columns_description': (columns_description.shape[0], columns_description.shape[1], nan_counts['columns_description'])
}

# Création d'un DataFrame à partir du dictionnaire
df_summary = pd.DataFrame.from_dict(data_summary, orient='index', columns=['Nombre de lignes', 'Nombre de colonnes', 'Nombre de données manquantes'])

# Affichage du DataFrame récapitulatif
df_summary

,Nombre de lignes,Nombre de colonnes,Nombre de données manquantes
X_train,307511,122,9152465
X_test,48744,121,1404419
bureau,1716428,17,3939947
bureau_balance,27299925,3,0
pos_cash,10001358,8,52158
credit_card,3840312,23,5877356
previous_application,1670214,37,11109336
installments_payments,13605401,8,5810
columns_description,219,5,133


### 2.2 Description des datasets

In [41]:
### la description des colonnes est dans le dataframe 'columns_description' :
# Configuration des options d'affichage
pd.set_option('display.max_colwidth', None)  # Aucune limite de largeur pour la colonne, affiche tout le texte
pd.set_option('display.max_columns', None)  # Affiche toutes les colonnes du DataFrame
pd.set_option('display.width', 1000)  # Largeur totale utilisable pour l'affichage du DataFrame
pd.set_option('display.max_rows', None)  # Aucune limite sur le nombre de lignes affichées

columns_description

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,"Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)",NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
5,8,application_{train|test}.csv,FLAG_OWN_REALTY,Flag if client owns a house or flat,NaN
6,9,application_{train|test}.csv,CNT_CHILDREN,Number of children the client has,NaN
7,10,application_{train|test}.csv,AMT_INCOME_TOTAL,Income of the client,NaN
8,11,application_{train|test}.csv,AMT_CREDIT,Credit amount of the loan,NaN
9,12,application_{train|test}.csv,AMT_ANNUITY,Loan annuity,NaN


In [43]:
pd.set_option('display.max_columns', None)
X_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [55]:
X_train['ORGANIZATION_TYPE'].unique()

array(['Business Entity Type 3', 'School', 'Government', 'Religion',
       'Other', 'XNA', 'Electricity', 'Medicine',
       'Business Entity Type 2', 'Self-employed', 'Transport: type 2',
       'Construction', 'Housing', 'Kindergarten', 'Trade: type 7',
       'Industry: type 11', 'Military', 'Services', 'Security Ministries',
       'Transport: type 4', 'Industry: type 1', 'Emergency', 'Security',
       'Trade: type 2', 'University', 'Transport: type 3', 'Police',
       'Business Entity Type 1', 'Postal', 'Industry: type 4',
       'Agriculture', 'Restaurant', 'Culture', 'Hotel',
       'Industry: type 7', 'Trade: type 3', 'Industry: type 3', 'Bank',
       'Industry: type 9', 'Insurance', 'Trade: type 6',
       'Industry: type 2', 'Transport: type 1', 'Industry: type 12',
       'Mobile', 'Trade: type 1', 'Industry: type 5', 'Industry: type 10',
       'Legal Services', 'Advertising', 'Trade: type 5', 'Cleaning',
       'Industry: type 13', 'Trade: type 4', 'Telecom',
       'I

### 2.3 Variables d'intérêt

https://www.kaggle.com/code/codename007/home-credit-complete-eda-feature-importance